## Simulation of a QKD process invaded by Quantum Cloning Machine

In [57]:
import numpy as np
import pandas as pd

from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.credentials import load_account, get_token_authentication, get_basic_authentication
from quantuminspire.api import QuantumInspireAPI

In [58]:
print('Enter mail address')
email = input()

print('Enter password')
password = getpass()

server_url = r'https://api.quantum-inspire.com'
authentication = BasicAuthentication(email, password)
qi = QuantumInspireAPI(server_url, authentication)

Enter mail address
y.zhang-108@student.tudelft.nl
Enter password
········


In [60]:
qi.list_backend_types()

Backend type: Starmon-5, number of qubits: 5
Backend type: QX single-node simulator, number of qubits: 26
Backend type: QX single-node simulator SurfSara, number of qubits: 31
Backend type: Spin-2, number of qubits: 2


### BB84 protocol: Charles H. Bennett and Gilles Brassard (1984)

Bob generate the key and measurement bases

In [202]:
n_key=64
bitstring = np.random.randint(2,size=n_key)
bases_A=np.random.randint(2,size=n_key)

bases_B=np.random.randint(2,size=n_key)
bases_E=bases_B.copy() # assume quantum memory

The expection key length is 1/2 of the random bitstring.

In [203]:
sum(bases_A==bases_B)

33

If base is 0, measure in Z, otherwise measure in X.

(key,base): state

00: $|0\rangle$ 
10: $|1\rangle$ 
01: $|+\rangle$ 
11: $|-\rangle$ 


Alice prepare the qubit and send it to Bob.

Assume Eve have a Cloning Machine and a Quantum Memory

So she can clone the states and measure in the same base after Bob publish her measurement bases.

In [227]:
def QKD_QACM(key,A_base,B_base,E_base):
    
    # here the qubits used in circuit is 1,3 instead of 0,4, for lower error rate
    
    qasm ='''
    version 2.0

    qubits 5

    # initialize the state
    {0}
    
    #preparation
    Ry q[0], 1.107149
    #rewrite CNOT q[0],q[4] and CNOT q[4],q[0] usign nearest neighbors
    SWAP q[0],q[2] 
    CNOT q[2],q[4]

    Ry q[4], 0.729728
    CNOT q[4],q[2]
    SWAP q[0],q[2]

    Ry q[0], 0.463648

    #copying
    CNOT q[2], q[0]
    CNOT q[2], q[4]
    CNOT q[0], q[2]
    CNOT q[4], q[2]

    #Rotate back and measure
    {1}
    {2}
    '''
    if A_base:
        prep="prep_x q[2]\n"
        if key:
            prep+="    Z q[2]\n"
    else:
        prep="prep_z q[2]\n"
        if key:
            prep+="    X q[2]\n"
            
    meas="measure_x q[2]\n" if B_base else "measure_z q[2]"
    invasion="measure_x q[0]\n" if E_base else "measure_z q[0]"
    
    return qasm.format(prep,meas,invasion)

Check the predetermined subset of their remained key to detect a eavesdropper.

If UQCM used, we can expect the error rate of key is 1/6. 

In [228]:
N_shots=4096
backend_type = qi.get_backend_type_by_name('Starmon-5')
# backend_type = qi.get_backend_type_by_name('QX single-node simulator')

data=[]
index=0

for param in zip(bitstring,bases_A,bases_B,bases_E):
    qasm=QKD_QACM(*param)
    result = qi.execute_qasm(qasm, backend_type=backend_type, number_of_shots=N_shots)
    hist=result.get('histogram', {}).values()
    data.append(hist)
    index+=1
    
    print(str(index)+"/"+str(n_key))
    print("Execution time: ", result.get('execution_time_in_seconds',{}))
    print("key:{0},bases_Alice:{1},bases_Bob:{2},bases_Eve:{3}".format(*param))
    print(hist)

1/64
Execution time:  0.622592
key:1,bases_Alice:0,bases_Bob:1,bases_Eve:1
odict_values([0.20705689277899345, 0.32029540481400437, 0.17970459518599563, 0.2929431072210066])
2/64
Execution time:  0.622592
key:1,bases_Alice:1,bases_Bob:0,bases_Eve:0
odict_values([0.34606797320764077, 0.23220044653932026, 0.2061523195236914, 0.21557926072934755])
3/64
Execution time:  0.622592
key:0,bases_Alice:0,bases_Bob:0,bases_Eve:0
odict_values([0.5799815781393921, 0.15965612526865214, 0.18913110224132637, 0.07123119435062941])
4/64
Execution time:  0.622592
key:1,bases_Alice:1,bases_Bob:1,bases_Eve:1
odict_values([0.17568370986920334, 0.18311533888228299, 0.3727705112960761, 0.2684304399524376])
5/64
Execution time:  0.622592
key:1,bases_Alice:1,bases_Bob:1,bases_Eve:1
odict_values([0.18783422459893048, 0.17479946524064172, 0.3689839572192513, 0.26838235294117646])
6/64
Execution time:  0.622592
key:1,bases_Alice:1,bases_Bob:1,bases_Eve:1
odict_values([0.17623188405797102, 0.1820289855072464, 0.3715

49/64
Execution time:  0.622592
key:0,bases_Alice:1,bases_Bob:1,bases_Eve:1
odict_values([0.401727115716753, 0.33264248704663213, 0.16649395509499137, 0.09913644214162348])
50/64
Execution time:  0.622592
key:0,bases_Alice:1,bases_Bob:1,bases_Eve:1
odict_values([0.4042067916877851, 0.3261530663963507, 0.15813482007095794, 0.11150532184490623])
51/64
Execution time:  0.622592
key:1,bases_Alice:0,bases_Bob:1,bases_Eve:1
odict_values([0.20726338958180485, 0.3033749082905356, 0.185986793837124, 0.3033749082905356])
52/64
Execution time:  0.622592
key:1,bases_Alice:1,bases_Bob:1,bases_Eve:1
odict_values([0.1726643598615917, 0.17024221453287197, 0.3698961937716263, 0.28719723183391005])
53/64
Execution time:  0.622592
key:0,bases_Alice:1,bases_Bob:1,bases_Eve:1
odict_values([0.415428390768258, 0.325640214985773, 0.15997470755611762, 0.0989566866898514])
54/64
Execution time:  0.622592
key:0,bases_Alice:1,bases_Bob:0,bases_Eve:0
odict_values([0.3745068285280728, 0.1872534142640364, 0.20637329

In [229]:
#[bin(int(i))[2:].zfill(5) for i in hist.keys()]
data_sheet=DataFramet=pd.DataFrame(data=np.array(list(map(list,data))), columns=["key","bases","00","01","10","11"])
data_sheet.head()

,00,01,10,11
0,0.207057,0.320295,0.179705,0.292943
1,0.346068,0.232200,0.206152,0.215579
2,0.579982,0.159656,0.189131,0.071231
3,0.175684,0.183115,0.372771,0.268430
4,0.187834,0.174799,0.368984,0.268382


In [231]:
np.array(list(map(list,data)))

array([[0.20705689, 0.3202954 , 0.1797046 , 0.29294311],
       [0.34606797, 0.23220045, 0.20615232, 0.21557926],
       [0.57998158, 0.15965613, 0.1891311 , 0.07123119],
       [0.17568371, 0.18311534, 0.37277051, 0.26843044],
       [0.18783422, 0.17479947, 0.36898396, 0.26838235],
       [0.17623188, 0.18202899, 0.3715942 , 0.27014493],
       [0.58387924, 0.1592139 , 0.18313871, 0.07376816],
       [0.58052196, 0.15436028, 0.19096117, 0.07415659],
       [0.17796073, 0.17162761, 0.38568714, 0.26472451],
       [0.38267615, 0.17485456, 0.19877182, 0.24369748],
       [0.39039467, 0.17831669, 0.20827389, 0.22301474],
       [0.35910955, 0.22437024, 0.20767428, 0.20884593],
       [0.39989977, 0.13730895, 0.38687046, 0.07592082],
       [0.18616145, 0.18890719, 0.36436024, 0.26057111],
       [0.18425197, 0.17070866, 0.37291339, 0.27212598],
       [0.41894469, 0.3245391 , 0.16369994, 0.09281627],
       [0.39066707, 0.13550703, 0.39874364, 0.07508226],
       [0.38962295, 0.19078311,

In [232]:
data_sheet.to_excel("QKD_Simulation.xlsx")

Find the key by take same measurement bases

In [235]:
secure_key=bitstring[bases_A==bases_B]

In [236]:
prob=data_sheet[bases_A==bases_B]

Assume Alice and Bob preserve half of the key for secure check.

Find the probability of Eve get the key without being noticed.

In [237]:
def get_success_prob(bit_string,probs,p):
    '''
    prob: npndarray for probability distribution
    p: number of bit to be check in public channel
    '''
    res=1
    undetected=1
    
    for (k,p) in zip(key[:p],probs[:p]):
        res*=p[k*3] # with 00 or 11
        undetected*=p[k*3]+p[k+1] # k+1=01 or 10
    return res,undetected

In [248]:
probs=[]
for i in range(1,16):
    p=get_success_prob(secure_key,prob.values,i)
    print(p)

(0.5799815781393921, 0.7396377034080442)
(0.10189331530332366, 0.26538130440354024)
(0.019139051871814135, 0.09623620163029452)
(0.005170317781023412, 0.06175853635057161)
(0.0003814048149487507, 0.015866191907780002)
(2.8283679784550892e-05, 0.004206409248625315)
(5.033384432842812e-06, 0.0014705116562642015)
(1.3115545927424025e-06, 0.0009189688439476449)
(2.416565155131671e-07, 0.000326197759725668)
(1.0124071438218507e-07, 0.00024252274634403603)
(2.5975705204034735e-08, 0.00015295358863052152)
(1.0645488254350202e-08, 0.00011212743323171863)
(2.5473805521914113e-09, 7.065020571768466e-05)
(2.381619813193167e-10, 1.8600502926421095e-05)
(8.170069122581292e-11, 1.0529194097414363e-05)


In [ ]:
import matplotlib.pylab as plt

In [250]:
plt.plot(p)

NameError: name 'plt' is not defined